In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
import accelerate
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


In [4]:
from huggingface_hub import notebook_login

notebook_login()

### Word form transmission

In [ ]:
train = pd.read_csv('/content/real_train.tsv', sep='\t')
test = pd.read_csv('/content/real_dev.tsv', sep='\t')

In [ ]:
train = train.rename(columns={'Copying expression':'label', 'set down':'span', 'It set down on the Moon in 1970 .': 'text'})
train = train.drop('span', axis = 1)
test = test.rename(columns={'Word form transmission':'label', 'declining':'span', 'Several factors were linked to the declining .': 'text'})
test = test.drop('span', axis = 1)

In [ ]:
train.loc[train['label'] != 'Word form transmission', 'label'] = 0
train.loc[train['label'] == 'Word form transmission', 'label'] = 1
train = pd.concat([train.loc[train['label'] == 1], train.loc[train['label'] == 0]][0:700])
test.loc[test['label'] != 'Word form transmission', 'label'] = 0
test.loc[test['label'] == 'Word form transmission', 'label'] = 1
test = pd.concat([test.loc[test['label'] == 1], test.loc[test['label'] == 0]][0:200])

In [ ]:
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1316 [00:00<?, ? examples/s]

Map:   0%|          | 0/361 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("f1")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "Other", 1: "WFT"}
label2id = {"Other": 0, "WFT": 1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="L1-classifier-WFT",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.585148,0.647619
2,No log,0.500887,0.772603
3,No log,0.361509,0.850440
4,No log,0.312056,0.911950
5,No log,0.307889,0.911950


TrainOutput(global_step=415, training_loss=0.38067421051393074, metrics={'train_runtime': 308.7215, 'train_samples_per_second': 21.314, 'train_steps_per_second': 1.344, 'total_flos': 243215337353520.0, 'train_loss': 0.38067421051393074, 'epoch': 5.0})

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1715102042.3ffd7dbeaebb.225.0:   0%|          | 0.00/5.81k [00:00<?, ?B/s]

events.out.tfevents.1715102159.3ffd7dbeaebb.225.1:   0%|          | 0.00/6.85k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

events.out.tfevents.1715102632.3ffd7dbeaebb.225.2:   0%|          | 0.00/6.76k [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/L1-classifier-WFT/commit/29cbde3c11978ca490286ac651ca463a4e61c9d7', commit_message='End of training', commit_description='', oid='29cbde3c11978ca490286ac651ca463a4e61c9d7', pr_url=None, pr_revision=None, pr_num=None)

### Tense semantics

In [ ]:
train = pd.read_csv('/content/real_train.tsv', sep='\t')
test = pd.read_csv('/content/real_dev.tsv', sep='\t')
train = train.rename(columns={'Copying expression':'label', 'set down':'span', 'It set down on the Moon in 1970 .': 'text'})
train = train.drop('span', axis = 1)
test = test.rename(columns={'Word form transmission':'label', 'declining':'span', 'Several factors were linked to the declining .': 'text'})
test = test.drop('span', axis = 1)

In [ ]:
train.loc[train['label'] != 'Tense semantics', 'label'] = 0
train.loc[train['label'] == 'Tense semantics', 'label'] = 1
train = pd.concat([train.loc[train['label'] == 1], train.loc[train['label'] == 0]][0:700])
test.loc[test['label'] != 'Tense semantics', 'label'] = 0
test.loc[test['label'] == 'Tense semantics', 'label'] = 1
test = pd.concat([test.loc[test['label'] == 1], test.loc[test['label'] == 0]][0:200])

In [ ]:
test

,label,text
5,1,"Overall, we can see that graph which show temp..."
7,1,The most striking difference was found in the ...
10,1,Comparasing 2014 year and 2015 we can see that...
12,1,It becomes a state park in 1958 .
14,1,While Japan have the fall from 1940 to 1960 an...
...,...,...
245,0,The selection process is strongly monitored by...
246,0,On the first diagramm we can see that desktop'...
247,0,Some of this equipment transforms into hi-teck...
248,0,as addition compounded violations are hard to ...


In [ ]:
train = Dataset.from_pandas(train.sample(frac=1))
test = Dataset.from_pandas(test.sample(frac=1))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1277 [00:00<?, ? examples/s]

Map:   0%|          | 0/349 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("f1")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "Other", 1: "TenSem"}
label2id = {"Other": 0, "TenSem": 1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="L1-classifier-TenSem",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.339891,0.881459
2,No log,0.249119,0.905660
3,No log,0.375092,0.905660
4,No log,0.312193,0.904459
5,No log,0.311711,0.914286


TrainOutput(global_step=400, training_loss=0.22951385498046875, metrics={'train_runtime': 318.0, 'train_samples_per_second': 20.079, 'train_steps_per_second': 1.258, 'total_flos': 268135859850840.0, 'train_loss': 0.22951385498046875, 'epoch': 5.0})

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1715103056.3ffd7dbeaebb.225.3:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/L1-classifier-TenSem/commit/8a418275c7a41b2e9695f7cf1cc43ea621ca3798', commit_message='End of training', commit_description='', oid='8a418275c7a41b2e9695f7cf1cc43ea621ca3798', pr_url=None, pr_revision=None, pr_num=None)

### Synonyms

In [81]:
train = pd.read_csv('/content/real_train.tsv', sep='\t')
test = pd.read_csv('/content/real_dev.tsv', sep='\t')
train = train.rename(columns={'Copying expression':'label', 'set down':'span', 'It set down on the Moon in 1970 .': 'text'})
train = train.drop('span', axis = 1)
test = test.rename(columns={'Word form transmission':'label', 'declining':'span', 'Several factors were linked to the declining .': 'text'})
test = test.drop('span', axis = 1)

In [82]:
train.loc[train['label'] != 'Synonyms', 'label'] = 0
train.loc[train['label'] == 'Synonyms', 'label'] = 1
train = pd.concat([train.loc[train['label'] == 1], train.loc[train['label'] == 0]][0:700])
test.loc[test['label'] != 'Synonyms', 'label'] = 0
test.loc[test['label'] == 'Synonyms', 'label'] = 1
test = pd.concat([test.loc[test['label'] == 1], test.loc[test['label'] == 0]][0:200])

In [83]:
train = Dataset.from_pandas(train.sample(frac=1))
test = Dataset.from_pandas(test.sample(frac=1))

In [84]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [85]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [86]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/2912 [00:00<?, ? examples/s]

Map:   0%|          | 0/726 [00:00<?, ? examples/s]

In [87]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [88]:
accuracy = evaluate.load("f1")

In [89]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [90]:
id2label = {0: "Other", 1: "Syn"}
label2id = {"Other": 0, "Syn": 1}

In [91]:
model = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="L1-classifier-Synonyms",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.630649,0.412371
2,No log,0.554122,0.641026
3,No log,0.591226,0.689655
4,No log,0.526143,0.724409
5,No log,0.537820,0.730924


TrainOutput(global_step=365, training_loss=0.47665451258829195, metrics={'train_runtime': 223.1584, 'train_samples_per_second': 26.17, 'train_steps_per_second': 1.636, 'total_flos': 213078843739200.0, 'train_loss': 0.47665451258829195, 'epoch': 5.0})

In [18]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

events.out.tfevents.1715104137.adaac59f58e7.373.0:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/L1-classifier-Synonyms/commit/4020a7636ef691df5e56ab92e3295e9943ef7800', commit_message='End of training', commit_description='', oid='4020a7636ef691df5e56ab92e3295e9943ef7800', pr_url=None, pr_revision=None, pr_num=None)

### Copying expression

In [61]:
train = pd.read_csv('/content/real_train.tsv', sep='\t')
test = pd.read_csv('/content/real_dev.tsv', sep='\t')
train = train.rename(columns={'Copying expression':'label', 'set down':'span', 'It set down on the Moon in 1970 .': 'text'})
train = train.drop('span', axis = 1)
test = test.rename(columns={'Word form transmission':'label', 'declining':'span', 'Several factors were linked to the declining .': 'text'})
test = test.drop('span', axis = 1)

In [62]:
train.loc[train['label'] != 'Copying expression', 'label'] = 0
train.loc[train['label'] == 'Copying expression', 'label'] = 1
train = pd.concat([train.loc[train['label'] == 1], train.loc[train['label'] == 0]][0:700])
test.loc[test['label'] != 'Copying expression', 'label'] = 0
test.loc[test['label'] == 'Copying expression', 'label'] = 1
test = pd.concat([test.loc[test['label'] == 1], test.loc[test['label'] == 0]][0:200])

In [45]:
train = Dataset.from_pandas(train.sample(frac=1))
test = Dataset.from_pandas(test.sample(frac=1))

In [46]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [47]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [48]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/2912 [00:00<?, ? examples/s]

Map:   0%|          | 0/726 [00:00<?, ? examples/s]

In [49]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [50]:
accuracy = evaluate.load("f1")

In [51]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [52]:
id2label = {0: "Other", 1: "CopExp"}
label2id = {"Other": 0, "CopExp": 1}

In [53]:
model = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
training_args = TrainingArguments(
    output_dir="L1-classifier-CopExp",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.411020,0.657005
2,No log,0.383656,0.713568
3,0.134700,0.406262,0.697479
4,0.134700,0.383039,0.723982
5,0.134700,0.441185,0.711712


TrainOutput(global_step=910, training_loss=0.10849972819233988, metrics={'train_runtime': 563.8462, 'train_samples_per_second': 25.823, 'train_steps_per_second': 1.614, 'total_flos': 621024312854400.0, 'train_loss': 0.10849972819233988, 'epoch': 5.0})

In [59]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

events.out.tfevents.1715105887.adaac59f58e7.373.4:   0%|          | 0.00/7.08k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/L1-classifier-CopExp/commit/f8a2f0ecd2eb519eb7f279f14d5f7a74188bb615', commit_message='End of training', commit_description='', oid='f8a2f0ecd2eb519eb7f279f14d5f7a74188bb615', pr_url=None, pr_revision=None, pr_num=None)

### Transliteration

In [65]:
train = pd.read_csv('/content/real_train.tsv', sep='\t')
test = pd.read_csv('/content/real_dev.tsv', sep='\t')
train = train.rename(columns={'Copying expression':'label', 'set down':'span', 'It set down on the Moon in 1970 .': 'text'})
train = train.drop('span', axis = 1)
test = test.rename(columns={'Word form transmission':'label', 'declining':'span', 'Several factors were linked to the declining .': 'text'})
test = test.drop('span', axis = 1)

In [66]:
train.loc[train['label'] != 'Transliteration', 'label'] = 0
train.loc[train['label'] == 'Transliteration', 'label'] = 1
train = pd.concat([train.loc[train['label'] == 1], train.loc[train['label'] == 0]][0:700])
test.loc[test['label'] != 'Transliteration', 'label'] = 0
test.loc[test['label'] == 'Transliteration', 'label'] = 1
test = pd.concat([test.loc[test['label'] == 1], test.loc[test['label'] == 0]][0:200])

In [67]:
train = Dataset.from_pandas(train.sample(frac=1))
test = Dataset.from_pandas(test.sample(frac=1))

In [68]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [69]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [70]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/2912 [00:00<?, ? examples/s]

Map:   0%|          | 0/726 [00:00<?, ? examples/s]

In [71]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [72]:
accuracy = evaluate.load("f1")

In [73]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [74]:
id2label = {0: "Other", 1: "Transliteration"}
label2id = {"Other": 0, "Transliteration": 1}

In [75]:
model = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [77]:
training_args = TrainingArguments(
    output_dir="L1-classifier-Transliteration",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.105490,0.961957
2,No log,0.116120,0.964770


TrainOutput(global_step=364, training_loss=0.08429067213456709, metrics={'train_runtime': 205.8385, 'train_samples_per_second': 28.294, 'train_steps_per_second': 1.768, 'total_flos': 248401502921280.0, 'train_loss': 0.08429067213456709, 'epoch': 2.0})

In [78]:
trainer.push_to_hub()

events.out.tfevents.1715106769.adaac59f58e7.373.5:   0%|          | 0.00/5.52k [00:00<?, ?B/s]

events.out.tfevents.1715106940.adaac59f58e7.373.6:   0%|          | 0.00/5.97k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/L1-classifier-Transliteration/commit/9fed6639d1a4c9549ffd17863b9440ed8fe44419', commit_message='End of training', commit_description='', oid='9fed6639d1a4c9549ffd17863b9440ed8fe44419', pr_url=None, pr_revision=None, pr_num=None)